In [51]:
import time
start_time = time.time()
import cv2
import numpy as np
import math
import blowfish
from PIL import Image 
from Cryptodome.Cipher import AES

%matplotlib auto


Using matplotlib backend: Qt5Agg


In [52]:
image = cv2.imread('bird.jpg')
cv2.imshow('original', image)

In [53]:
img_str = cv2.imencode('.jpg', image)[1].tobytes()
if len(img_str)%8 != 0:#check for compatibility
    while len(img_str)%8 != 0:
        img_str += b'\x00'

og_img_np = np.frombuffer(img_str, dtype=np.uint8)
img_str2 = img_str.replace(b'\x00', b'\x01')
og_img_np2 = np.frombuffer(img_str2, dtype=np.uint8)

In [54]:
len(img_str)/8 #check for compatibility

1493.0

In [55]:
cipher_b = blowfish.Cipher(b'blowfishpasswordishere')

In [56]:
data = img_str # electronic codebook mode

data_encrypted = b"".join(cipher_b.encrypt_ecb(data))

In [57]:
img_str_np_enc_blow = np.frombuffer(data_encrypted, dtype=np.uint8)

In [58]:
a = len(img_str_np_enc_blow)
i = 1
while a%2 == 0:
    i = i*2
    a = a//2

img_str_np_enc_blow1 = img_str_np_enc_blow.reshape(i, len(img_str_np_enc_blow)//i)
enc_img = Image.fromarray(img_str_np_enc_blow)
enc_img.show(title='blowfish encrypted')

In [59]:
key = b'aespasswordisnow'
cipher_a = AES.new(key, AES.MODE_EAX)
nonce = cipher_a.nonce
ciphertext_enc, tag = cipher_a.encrypt_and_digest(data_encrypted)

In [60]:
img_str_np_enc_aes = np.frombuffer(ciphertext_enc, dtype=np.uint8)
img_str_np_enc_aes = np.frombuffer(ciphertext_enc, dtype=np.uint8)
a = len(img_str_np_enc_aes)
i = 1
while a%2 == 0:
    i = i*2
    a = a//2


img_str_np_enc_aes1 = img_str_np_enc_aes.reshape(i,len(img_str_np_enc_aes)//i)

In [61]:
enc_img = Image.fromarray(img_str_np_enc_aes1, 'L')
enc_img.show(title='aes encrypted')

In [62]:
cipher_a = AES.new(key, AES.MODE_EAX, nonce=nonce)
plaintext_enc = cipher_a.decrypt(ciphertext_enc)
try:
    cipher_a.decrypt(ciphertext_enc)
    print("AES decryption successful")
except ValueError:
    print("Decryption failed")

AES decryption successful


In [63]:
data_decrypted = b"".join(cipher_b.decrypt_ecb(plaintext_enc))

#assert data == data_decrypted

In [64]:
img_str_np = np.frombuffer(data_decrypted, dtype=np.uint8)

In [65]:
img = cv2.imdecode(img_str_np, cv2.IMREAD_COLOR)

In [66]:
cv2.imshow('decrypted', img)

In [67]:
def psnr(img1, img2):
    mse = np.mean((img1 - img2) ** 2)
    if mse == 0:
        return 100
    PIXEL_MAX = 255.0
    return 20 * math.log10(PIXEL_MAX / math.sqrt(mse))

d = psnr(image, img)
print(d)

43.22586472210333


In [68]:
cv2.PSNR(image, img)

42.722213649793986

In [69]:
MSE = np.square(np.subtract(image,img)).mean()
MSE

3.0938117793787896

In [70]:
np.corrcoef(image.ravel(), img.ravel())

array([[1.        , 0.99943421],
       [0.99943421, 1.        ]])

In [71]:
print("--- %s seconds ---" % (time.time() - start_time))

--- 7.54942774772644 seconds ---


In [72]:
def entropy(array):
    total_entropy = 0
    for i in array:
        total_entropy += -i * math.log(i, 2)
    return total_entropy

In [73]:
#entropy(img_str_np_enc)